# Interpreter Coding Exercise

You are asked to write an expression processor for simple numeric expressions with the following constraints:

Expressions use integral values (e.g., `'13'`), single-letter variables defined in Variables, as well as + and - operators only

There is no need to support braces or any other operations

If a variable is not found in `variables` (or if we encounter a variable with >1 letter, e.g. ab), the evaluator returns 0 (zero)

In case of any parsing failure, evaluator returns 0

Example:

`calculate("1+2+3")`  should return 6

`calculate("1+2+xy")`  should return 0

`calculate("10-2-x")`  when x=3 is in variables  should return 5

In [1]:
import re
import unittest
from enum import Enum

In [2]:
def megasplit(pattern, string):
    splits = list((m.start(), m.end()) for m in re.finditer(pattern, string))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits] + [len(string)]
    return [string[start:end] for start, end in zip(starts, ends)]

In [3]:
class ExpressionProcessor:
    class NextOp(Enum):
        PLUS = 1
        MINUS = 2

    def __init__(self):
        self.variables = {}

    def calculate(self, expression):
        current = 0
        next_op = None

        # doesn't work in python 3.5
        # parts = re.split('(?<=[+-])', expression)
        parts = megasplit('(?<=[+-])', expression)

        for part in parts:
            noop = re.split('[\+\-]', part)
            first = noop[0]
            value = 0

            try:
                value = int(first)
            except ValueError:
                if len(first) == 1 and first[0] in self.variables:
                    value = self.variables[first[0]]
                else:
                    return 0

            if not next_op:
                current = value
            elif next_op == self.NextOp.PLUS:
                current += value
            elif next_op == self.NextOp.MINUS:
                current -= value

            if part.endswith('+'):
                next_op = self.NextOp.PLUS
            elif part.endswith('-'):
                next_op = self.NextOp.MINUS

        return current

In [4]:
ep = ExpressionProcessor()
ep.variables['x'] = 5

print(ep.calculate('1'))
assert ep.calculate('1') == 1

print(ep.calculate('1+2'))
assert ep.calculate('1+2') == 3

print(ep.calculate('1+x'))
assert ep.calculate('1+x') == 6

print(ep.calculate('1+xy'))
assert ep.calculate('1+xy') == 0

1
3
6
0


In [5]:
class FirstTestSuite(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        ep = ExpressionProcessor()
        ep.variables['x'] = 5
        cls.ep = ep

    def test_simple(self):
        self.assertEqual(1, self.ep.calculate('1'))

    def test_addition(self):
        self.assertEqual(3, self.ep.calculate('1+2'))

    def test_addition_with_variable(self):
        self.assertEqual(6, self.ep.calculate('1+x'))

    def test_failure(self):
        self.assertEqual(0, self.ep.calculate('1+xy'))

unittest.main(argv=['ignored', '-v'], exit=False)

test_addition (__main__.FirstTestSuite) ... ok
test_addition_with_variable (__main__.FirstTestSuite) ... ok
test_failure (__main__.FirstTestSuite) ... ok
test_simple (__main__.FirstTestSuite) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK
